In [1]:
import pandas as pd
from utils.openai_query import openai_chat
import re
import json

In [30]:
with open('jsonFiles/OmicsRunLLM.json', 'r') as f:
    config = json.load(f)
context = config['CONTEXT']
max_tokens = 2000
rate_per_token = config['RATE_PER_TOKEN']
LOG_FILE = config['LOG_NAME']
DOLLAR_LIMIT = config['DOLLAR_LIMIT']


In [25]:
# Read the results of processing the 300 'omics datasets
temp = pd.read_csv("data/omics_revamped_LLM_Enrichr_simVals_failure_by_apv_best_JI.tsv", delimiter="\t")
input_analyses = temp[temp["LLM Name"] != "System of unrelated proteins"]
print(len(temp[temp['Score']>=0.82]))
print(len(input_analyses))
input_analyses.head(3)

133
135


,Unnamed: 0.1,Unnamed: 0,Source,GeneSetID,GeneSetName,GeneList,n_Genes,LLM Name,LLM Analysis,Score,...,Genes,Term,GO term,GO ID,GO_term_genes,LLM_name_GO_term_sim,enrichr_JI,coverage,LLM_success_TF,enrichr_success_TF
0,7148,0,L1000,BRD-A00546892_-666_MCF7_6.0_h_10.0_um,BRD-A00546892 -666 MCF7 6.0 h 10.0 um,CITED2 COL5A1 CRABP2 KCTD12 MDFIC MMP2 NRP1 OR...,24,Cellular Matrix Remodeling and Tissue Development,"1. CITED2, TWIST1, and LMO2 are transcriptiona...",0.85,...,CITED2;TWIST1,Regulation Of Peroxisome Proliferator Activate...,Regulation Of Peroxisome Proliferator Activate...,GO:0035358,ASXL2 ASXL1 PTGIS BMP2 CITED2 GPS2 LMO3 PLIN5 ...,0.245878,0.062500,0.083333,True,False
1,7149,0,L1000,BRD-A00993607_ALPRENOLOL_MCF7_6.0_h_10.0_um,BRD-A00993607 ALPRENOLOL MCF7 6.0 h 10.0 um,1060P11.3 ADM AHR AMIGO2 ARL4C ATP10D CAV2 CD4...,47,Cellular Adhesion and Extracellular Matrix Int...,"1. Several proteins in this system, such as CD...",0.85,...,NT5E;HPRT1,Purine Ribonucleoside Monophosphate Catabolic ...,Purine Ribonucleoside Monophosphate Catabolic ...,GO:0009169,PNP NT5C1A NT5E HPRT1 SORD PRTFDC1 NT5C2,0.166891,0.038462,0.042553,True,False
5,7374,0,L1000,BRD-A01593789_CHLORMADINONE ACETATE_MCF7_6.0_h...,BRD-A01593789 CHLORMADINONE ACETATE MCF7 6.0 h...,1060P11.3 ADH5 ADM ATF3 CASP1 CLEC2B CPA3 CRIM...,38,Stress Response and Apoptosis Regulation,"1. ATF3, Activating Transcription Factor 3, is...",0.85,...,TNC;FOLR1,Axon Regeneration (GO:0031103),Axon Regeneration,GO:0031103,GAP43 RTN4RL1 RTN4RL2 TNC NREP MTR MAPK8IP3 FO...,0.306703,0.044444,0.052632,True,False


In [31]:
count_genes_prompt_template_old = """
The provided text is an analysis of a gene set to describe its common functions and to provide a name for the gene set reflecting the predominant function or functions.

Do not critique the analysis or the name. Your job is to find the genes that support the name

You are provided with this list of the gene symbols mentioned in the text. Only consider these genes.

<genes mentioned in the text>{genes_in_text}</genes mentioned in the text>
 
For each gene from that list mentioned in a sentence or paragraph, decide whether the text is making a *definite* assertion about the gene that supports the name.
DO NOT make your own assesments about the gene's function or the validity of the analysis, just evaluate the text.
<example of definite assertion>
"XRCC1 is involved in the DNA damage response"
</example of definite assertion>
<example of non-definite assertion>
"E2F1 may be involved in homolougus recombination."
"SREBF1 could be related to RAD51's function."
</example of non-definite assertion>

As you work, briefly explain your reasoning for each gene symbol that supports the name.
<example of explanation>
Reasoning:

1. CITED2: Involved in transcriptional regulation influencing gene expression, which is crucial for tissue development.
2. TWIST1: Crucial for mesodermal tissue development and epithelial-mesenchymal transition (EMT), directly supporting tissue development.
3. LMO2: Implicated in hematopoiesis and vascular development, supporting tissue development.
4. COL5A1: Encodes a component of type V collagen, essential for connective tissue structure and function, supporting tissue development and extracellular matrix remodeling.
</example of explanation>

For each gene symbol that supports the name, add it to your list of supporting genes.

<text>{text}</text>

<name>{name}</name>

Output the supporting genes and non-supporting genes in the following format: 

<format>
-- Explanation --
<explanation of the reasoning for the supporting genes>
-- genes supporting the name: <list of gene symbols of genes supporting the name>
</format>
"""

count_genes_prompt_template = """
Analyze the provided text, which describes a gene set's common functions and suggests a name reflecting its predominant function(s). Your task is to identify genes that support this name based solely on the information given in the text.

Context: Gene sets are groups of genes that share common biological functions, pathways, or other characteristics. Naming these sets based on their predominant functions helps researchers quickly understand their significance.

Input:
1. A list of gene symbols in the gene set, provided in comma-separated format:
<gene set>{genes_in_text}</gene set>

2. The analysis text:
<text>{text}</text>

3. The suggested name for the gene set:
<name>{name}</name>

Instructions:
1. Evaluate each gene from the provided list that is mentioned in the text.
2. Determine if the text makes a definite assertion about the gene that supports the given name.
   - A definite assertion clearly states a gene's function or role without using speculative language.
   - Example of a definite assertion: "XRCC1 is involved in the DNA damage response"
   - Example of a non-definite assertion: "E2F1 may be involved in homologous recombination"
3. If a gene is mentioned multiple times, consider the strongest assertion made about it.
4. In case of contradictory statements about a gene, favor the most recent or specific assertion.
5. For each gene you determine supports the name:
   - Briefly explain your reasoning (max 50 words per gene)
   - Assign a confidence level (High, Medium, Low) based on the strength of the assertion
6. Handle acronyms or alternative gene names as equivalent to official gene symbols.
7. If no genes seem to support the name or if all genes support it, state this observation.

Output your analysis in the following format:

-- Summary --
[Provide a brief summary (max 100 words) of why the selected genes support the given name]

-- Explanation --
[Gene Symbol]: [Confidence Level]
[Explanation of reasoning (max 50 words)]

[Repeat for each supporting gene]

-- genes supporting the name: [List of gene symbols of genes supporting the name]

Do not critique the analysis or the name. Base your evaluation solely on the information provided in the text.
"""

def parse_analysis(text):
    formatted_genes = ""
    supporting_count = 0
    try:
        supporting_genes = re.search(r"genes supporting the name: (.+)", text).group(1).strip("[]").split(", ")
        supporting_count = len(supporting_genes)
        formatted_genes = " ".join(supporting_genes)
    except Exception as e:
        print(text)

    return formatted_genes, supporting_count

def get_genecount_mentioned_in_text(genelist, text):
    genes = set(genelist.split(" "))
    number_mentioned = sum(1 for gene in genes if gene in text)
    return number_mentioned

def get_genes_supporting_name(name, text, prompt_template, genes_in_text):
    prompt = prompt_template.format(genes_in_text=genes_in_text, text=text, name=name)
    temperature = 0.0
    model = "gpt-4o"
    analysis, finger_print  = openai_chat(context, prompt, model, temperature, max_tokens, rate_per_token, LOG_FILE, DOLLAR_LIMIT)
    # print(analysis)
    supporting_genes, supporting_count = parse_analysis(analysis)
    return supporting_genes, supporting_count, analysis

def process_gene_set_analyses(input_analyses, prompt_template, number_to_process=None, progress_file="data/supporting_gene_counts_progress.tsv"):
    if number_to_process is not None:
        analyses = input_analyses.head(number_to_process).copy()
    else:
        analyses = input_analyses.copy()
    
    print(f"Processing {len(analyses)} analyses")

    def get_mentioned(row): 
        return get_genecount_mentioned_in_text(row["GeneList"], row["LLM Analysis"])

    def get_supporting(row):
        print(f'processing {row["GeneSetName"]}...')
        return get_genes_supporting_name(row["LLM Name"], row["LLM Analysis"], prompt_template, row["GeneList"])
    
    #initialize columns
    analyses["Supporting Genes"] = None
    analyses["Supporting Count"] = None
    analyses["LLM Support Analysis"] = None
    analyses["GenesMentionedInText"] = None

    for idx, row in analyses.iterrows():
        supporting_genes, supporting_count, supporting_analysis = get_supporting(row)
        mentioned_genes = get_mentioned(row)
        analyses.loc[idx, "Supporting Genes"] = supporting_genes
        analyses.loc[idx, "Supporting Count"] = supporting_count
        analyses.loc[idx, "LLM Support Analysis"] = supporting_analysis
        analyses.loc[idx, "GenesMentionedInText"] = mentioned_genes
        # analyses[["Supporting Genes", "Supporting Count", "LLM Support Analysis"]] = analyses.apply(
        #     lambda row: pd.Series(get_supporting(row)), axis=1)

        results = analyses[[
            "Source", "GeneSetID", "GeneSetName", "GeneList", "n_Genes", "Supporting Genes", 
            "Supporting Count", 
            "LLM Support Analysis", "GenesMentionedInText"]].copy()
        results.to_csv(progress_file, sep='\t', index=False)
    
    return results

In [32]:
results_df = process_gene_set_analyses(input_analyses, count_genes_prompt_template)
results_df.head()

Processing 135 analyses
processing BRD-A00546892 -666 MCF7 6.0 h 10.0 um...
1474
processing BRD-A00993607 ALPRENOLOL MCF7 6.0 h 10.0 um...
1852
processing BRD-A01593789 CHLORMADINONE ACETATE MCF7 6.0 h 10.0 um...
1588
processing BRD-A02006392 NITRENDIPINE MCF7 6.0 h 10.0 um...
1745
processing BRD-A02176148 TUBAIC ACID MCF7 6.0 h 10.0 um...
1715
processing BRD-A04756508 NORGESTIMATE MCF7 6.0 h 10.0 um...
2044
processing BRD-A06352418 TERFENADINE MCF7 6.0 h 10.0 um...
2010
processing BRD-A08003242 RHODOMYRTOXIN B MCF7 6.0 h 10.0 um...
2097
processing BRD-A09056319 -666 MCF7 6.0 h 10.0 um...
1610
processing BRD-A09539288 HOMATROPINE BROMIDE MCF7 6.0 h 10.0 um...
1750
processing BRD-A10715913 -666 MCF7 6.0 h 10.0 um...
1550
processing BRD-A11605036 -666 MCF7 6.0 h 10.0 um...
1776
processing BRD-A11678676 -666 MCF7 6.0 h 10.0 um...
1744
processing BRD-A13914662 -666 MCF7 6.0 h 10.0 um...
1825
processing BRD-A13964793 -666 MCF7 6.0 h 10.0 um...
1528
processing BRD-A15010982 10006350 MCF7 6.0

,Source,GeneSetID,GeneSetName,GeneList,n_Genes,Supporting Genes,Supporting Count,LLM Support Analysis,GenesMentionedInText
0,L1000,BRD-A00546892_-666_MCF7_6.0_h_10.0_um,BRD-A00546892 -666 MCF7 6.0 h 10.0 um,CITED2 COL5A1 CRABP2 KCTD12 MDFIC MMP2 NRP1 OR...,24,CITED2 TWIST1 COL5A1 MMP2 NRP1 VIM,6,-- Summary --\nThe selected genes support the ...,24
1,L1000,BRD-A00993607_ALPRENOLOL_MCF7_6.0_h_10.0_um,BRD-A00993607 ALPRENOLOL MCF7 6.0 h 10.0 um,1060P11.3 ADM AHR AMIGO2 ARL4C ATP10D CAV2 CD4...,47,CD44 EMP1 AMIGO2 POSTN COMP EXT1 CAV2 ARL4C CD...,15,-- Summary --\nThe selected genes support the ...,15
5,L1000,BRD-A01593789_CHLORMADINONE ACETATE_MCF7_6.0_h...,BRD-A01593789 CHLORMADINONE ACETATE MCF7 6.0 h...,1060P11.3 ADH5 ADM ATF3 CASP1 CLEC2B CPA3 CRIM...,38,ATF3 CASP1 ADM DKK3 MECOM PIK3R1 S100A8 SST TNC,9,-- Summary --\nThe selected genes support the ...,10
6,L1000,BRD-A02006392_NITRENDIPINE_MCF7_6.0_h_10.0_um,BRD-A02006392 NITRENDIPINE MCF7 6.0 h 10.0 um,1060P11.3 ADAM17 AKAP12 ALCAM AMFR BIK CASP2 C...,76,TP53 CASP2 BIK HSPA1A GLUL GPX3 CDKN2A FOSL1 A...,11,-- Summary --\nThe selected genes support the ...,11
7,L1000,BRD-A02176148_TUBAIC ACID_MCF7_6.0_h_10.0_um,BRD-A02176148 TUBAIC ACID MCF7 6.0 h 10.0 um,CASP1 CAST CD52 CLC COL1A2 COL3A1 COL5A2 CPA3 ...,34,COL1A2 COL3A1 COL5A2 FN1 LUM SPON1 GJA1 GPNMB ...,14,-- Summary --\nThe selected genes support the ...,15


In [33]:
# apply a function to calculate the percent of genes supporting the name to each row and add new columns to the DataFrame

results_df.loc[:,'LLM_coverage'] = results_df.apply(lambda row: row['Supporting Count'] / row['n_Genes'], axis=1)

results_df

,Source,GeneSetID,GeneSetName,GeneList,n_Genes,Supporting Genes,Supporting Count,LLM Support Analysis,GenesMentionedInText,LLM_coverage
0,L1000,BRD-A00546892_-666_MCF7_6.0_h_10.0_um,BRD-A00546892 -666 MCF7 6.0 h 10.0 um,CITED2 COL5A1 CRABP2 KCTD12 MDFIC MMP2 NRP1 OR...,24,CITED2 TWIST1 COL5A1 MMP2 NRP1 VIM,6,-- Summary --\nThe selected genes support the ...,24,0.250000
1,L1000,BRD-A00993607_ALPRENOLOL_MCF7_6.0_h_10.0_um,BRD-A00993607 ALPRENOLOL MCF7 6.0 h 10.0 um,1060P11.3 ADM AHR AMIGO2 ARL4C ATP10D CAV2 CD4...,47,CD44 EMP1 AMIGO2 POSTN COMP EXT1 CAV2 ARL4C CD...,15,-- Summary --\nThe selected genes support the ...,15,0.319149
5,L1000,BRD-A01593789_CHLORMADINONE ACETATE_MCF7_6.0_h...,BRD-A01593789 CHLORMADINONE ACETATE MCF7 6.0 h...,1060P11.3 ADH5 ADM ATF3 CASP1 CLEC2B CPA3 CRIM...,38,ATF3 CASP1 ADM DKK3 MECOM PIK3R1 S100A8 SST TNC,9,-- Summary --\nThe selected genes support the ...,10,0.236842
6,L1000,BRD-A02006392_NITRENDIPINE_MCF7_6.0_h_10.0_um,BRD-A02006392 NITRENDIPINE MCF7 6.0 h 10.0 um,1060P11.3 ADAM17 AKAP12 ALCAM AMFR BIK CASP2 C...,76,TP53 CASP2 BIK HSPA1A GLUL GPX3 CDKN2A FOSL1 A...,11,-- Summary --\nThe selected genes support the ...,11,0.144737
7,L1000,BRD-A02176148_TUBAIC ACID_MCF7_6.0_h_10.0_um,BRD-A02176148 TUBAIC ACID MCF7 6.0 h 10.0 um,CASP1 CAST CD52 CLC COL1A2 COL3A1 COL5A2 CPA3 ...,34,COL1A2 COL3A1 COL5A2 FN1 LUM SPON1 GJA1 GPNMB ...,14,-- Summary --\nThe selected genes support the ...,15,0.411765
...,...,...,...,...,...,...,...,...,...,...
272,Viral_Infections,Dhori Virus_48Hour,Dhori Virus 48Hour,KLK11 ADAMTS18 GOLGA8I ZNF71 LOC283745 NMUR2 P...,93,IFIH1 STAT2 IRF1 TLR3 CD274 ISG20 IFIT2 CXCL11...,11,-- Summary --\nThe gene set predominantly supp...,23,0.118280
275,Viral_Infections,Ebolavirus(ZEBOV)_6day_PBMCs_rhAPCtreated,Ebolavirus(ZEBOV) 6day PBMCs rhAPCtreated,WDR16 GNG12 NSMAF,3,WDR16 GNG12 NSMAF,3,"-- Summary --\nThe gene set ""Intracellular Sig...",3,1.000000
291,Viral_Infections,SARS-dORF6_72Hour...135,SARS-dORF6 72Hour...135,LPAR3 FLCN ZNF79 KIF2A RDH11 MCM8 LAPTM4A LEPR...,72,MCM8 RPA1 POLR3D NFKB2 PPP1R15A HES1 RPS6P6 RP...,10,"-- Summary --\nThe gene set ""Cellular Prolifer...",18,0.138889
294,Viral_Infections,hMPV_24Hour,hMPV 24Hour,SECISBP2 BICC1 NEDD9 MAGEA6 TM4SF18 PLP2 EIF2A...,72,IRF1 IRF7 IRF9 ISG15 IFI35 TRIM21 TRIM22 TRIM3...,21,-- Summary --\nThe selected genes support the ...,21,0.291667


In [44]:
# check if there is None in the Supporting Genes column
results_df = results_df.fillna(0)
results_df[results_df['Supporting Count']==0]
results_df['Supporting Count'] = pd.to_numeric(results_df['Supporting Count'], errors='coerce')
results_df['GenesMentionedInText'] = pd.to_numeric(results_df['GenesMentionedInText'], errors='coerce')
(results_df['Supporting Count'] <= results_df['GenesMentionedInText']).sum()

133

In [43]:
results_df[results_df['Supporting Count'] > results_df['GenesMentionedInText']]

,Source,GeneSetID,GeneSetName,GeneList,n_Genes,Supporting Genes,Supporting Count,LLM Support Analysis,GenesMentionedInText,LLM_coverage
151,NeST,Cluster2-107,Cluster2-107,HOXA1 HOXA2 HOXA3 HOXA4 HOXA5 HOXA6 HOXA11 HOX...,21,HOXA1 HOXA2 HOXA3 HOXA4 HOXA5 HOXA6 HOXA11 HOX...,21,-- Summary --\nThe provided text describes the...,8,1.0
238,NeST,Cluster2-191,Cluster2-191,APC2 CELSR1 CELSR2 CELSR3 FAT4 FREM2 FUZ FZD1 ...,22,APC2 CELSR1 CELSR2 CELSR3 FAT4 FREM2 FUZ FZD1 ...,22,-- Summary --\nThe selected genes support the ...,14,1.0


In [34]:
results_df.to_csv("data/omics_revamped_LLM_Enrichr_simVals__genecounts_DF.tsv", sep="\t", index=False)